In [1]:
import pandas as pd
import requests
from bs4 import BeautifulSoup as bs
import re
from datetime import date 


# setting up match day 
t = date.today() 
today = t.strftime('%d/%m/%Y')
today



'24/11/2020'

In [8]:
class PremireLeague:
    
    def __init__(self, x):
        
        self.x = x
        
        self.link = 'https://www.premierleague.com/match/{}'.format(self.x)

        self.r = requests.get(self.link)

        self.soup = bs(self.r.content, 'html.parser')
        


In [9]:
    

######## home team ###############
    
class HomeTeam(PremireLeague):
    
    def __init__(self, x):
        
        super().__init__(x)

    def home_goal(self):

        H_Goal = []

        goals = self.soup.select('div.matchEvents div.home div.event')

        for goal in goals:

            name = goal.a.text


            try:
                action = goal.select('div.icnContainer span')[0].text

                name = action + ' - ' + name

                H_Goal.append(name)

            except IndexError:

                break

        return H_Goal
    
    def goal_no_home(self):
        
        goal_no_home = []
        
        goals = self.soup.select('div.matchEvents div.home div.event')
        
        pattern = re.compile(r"\d+") # to get time of goal
        
        for goal in goals:
            
            matches = pattern.findall(goal.text)

            goal_no_home.append(len(matches))
        
        return goal_no_home
        
# Home team assists

    def home_assist(self):

        H_Assist = []

        assists = self.soup.select('div.assists div.home div.event')

        for assist in assists:

            name = assist.a.text

            H_Assist.append(name)

        return H_Assist

# Home team name

    def home_name(self):

        home_name = self.soup.select('div.home')[0].select('span.long')[0].text

        return home_name
        
    def home(self):

        print(self.home_name() + ':- \n')

        print(f'{self.home_goal()} \n')

        print("Assist Given By:- \n")

        print(f'{self.home_assist()} \n')



In [10]:

########### Away Team #########


class AwayTeam(PremireLeague):
    
    def __init__(self, x):
        
        super().__init__(x)

#  Goal scored away team

    def away_goal(self):
        
        A_Goal = []

        goals = self.soup.select('div.matchEvents div.away div.event')

        for goal in goals:

            name = goal.a.text

            try:
                action = goal.select('div.icnContainer span')[0].text

                name = action + ' - ' + name

                A_Goal.append(name)

            except IndexError:

                break
                
        return A_Goal

# no of goals per player of away team 

    def goal_no_away(self):
        
        goal_no_away = []
        
        goals = self.soup.select('div.matchEvents div.away div.event')
        
        pattern = re.compile(r"\d+") # to get time of goal
        
        for goal in goals:
            
            matches = pattern.findall(goal.text)

            goal_no_away.append(len(matches))
        
        return goal_no_away


# Away team assists

    def away_assist(self):
        
        A_Assist = []

        assists = self.soup.select('div.assists div.away div.event')

        for assist in assists:

            name = assist.a.text

            A_Assist.append(name)

        return A_Assist
    
# Away team name

    def away_name(self):

        away_name = self.soup.select('div.away')[0].select('span.long')[0].text

        return away_name 

    def away(self):
        
        
        print(self.away_name() + ':- \n')

        print(f'{self.away_goal()} \n')

        print("Assist Given By:- \n")

        print(f'{self.away_assist()} \n')
    

In [68]:

############## Result#####################


class MatchResult(HomeTeam, AwayTeam):
    
    def __init__(self, x):
        
        super().__init__(x)
        
    def result(self):
        
        try:
            goal = self.soup.select('div.score')[0]

            print(f'{self.home_name()} - {self.away_name()}:- \n')

            print(goal.text)
        
        except IndexError:
        
            print(f'Match upcoming \n')

   


In [69]:
import numpy as np    
########## Making csv/Excel files ############

class Visualization(HomeTeam, AwayTeam):
    
    def __init__(self, x):
        
        super().__init__(x)
        
    def home_file(self):
        
   
        home_goal = self.home_goal() 
    
        home_goal_no = self.goal_no_home()
        
        home_assist = self.home_assist()

        
        if len(home_goal) > 0:   # filtering red cards from home team
            for i in range(len(home_goal)): 
                f = re.compile(r'(Red Card)')
                l = f.findall(home_goal[i])
                if l:
                    home_goal[i] = np.nan
                    home_goal_no[i] = np.nan
        
        if len(home_goal) == 0:    # for no score home team
            home_goal = [np.nan]
            home_assist = [np.nan]
            home_goal_no = [np.nan]
                 
            
    
        df_home_goal = pd.DataFrame({self.home_name()+' - goal': home_goal,
                                     'Goals_no': home_goal_no})
                                    
        
        df_home_assist = pd.DataFrame({self.home_name()+' - assists': home_assist})
        
        df_home = pd.concat([df_home_goal,
                             df_home_assist], axis=1)
        
        return df_home
        
    def away_file(self):
             
        away_goal = self.away_goal()
        
        away_assist = self.away_assist()
        
        away_goal_no = self.goal_no_away()
        
        
        if len(away_goal) > 0:   # filtering red cards from away team
            for i in range(len(away_goal)): 
                f = re.compile(r'(Red Card)')
                l = f.findall(away_goal[i])
                if l:
                    away_goal[i] = np.nan
                    away_goal_no[i] = np.nan

        
        if len(away_goal) == 0:    # for no score away team
            away_goal = [np.nan]
            away_assist = [np.nan]
            away_goal_no = [np.nan]
                        
            
            
        
        df_away_goal = pd.DataFrame({self.away_name()+' - goal': away_goal,
                                     'Goals_no': away_goal_no})
        
        df_away_assist = pd.DataFrame(away_assist,
                                      columns=[self.away_name()+' - assists'])
        
        df_away = pd.concat([df_away_goal,
                             df_away_assist], axis=1)
        
        return df_away
    

In [56]:
# output match result epl offc site

score = MatchResult(58982)
score.result()
saving = Visualization(58982)

df = saving.away_file()

# adding date time index in dataframe

#df['date'] = [today for i in range(df.shape[0])]
#df.set_index('date', inplace=True)


df

# Updating files for each team(due)

#df.to_csv('Chelsea.csv')

Newcastle United - Chelsea:- 

0-2


In [81]:
# Getting all matches in this week and id code epl offc site 

def completed_match():
    link = 'https://www.premierleague.com/'

    r = requests.get(link)

    soup = bs(r.content, 'html.parser')

    number = soup.select('a.matchAbridged.form.embeddableMatchContainer')
    
    Team_code = []
    for tag in number:

        text = tag.get('href')
        c = tag.select('span.teamName')
        T = []
        for team_1 in c:

            team = team_1.abbr.get('title')
            T.append(team)



        pattern = re.compile(r'\d+')
        code = pattern.findall(text)
        T.append(code)

        Team_code.append(T)
        return Team_code
    

CompletedMatch = completed_match()

if not CompletedMatch:
    print('No Match completed this updated week \n visit:- https://www.premierleague.com/')
else:
    completed_match()

No Match completed this updated week 
 visit:- https://www.premierleague.com/


In [80]:
# weekly fixtures matches with code premire league official site

def weekly_fixtures():
    link = 'https://www.premierleague.com/'

    r = requests.get(link)
    soup = bs(r.content, 'html.parser')

    fixtures = soup.select('a.matchAbridged.embeddableMatchContainer')
    Team_code = []
    for fixture in fixtures:
        text = fixture.get('href')
        c = fixture.select('span.teamName')
        T = []
        for team_1 in c:

            team = team_1.abbr.get('title')
            T.append(team)
            


        pattern = re.compile(r'\d+')
        code = pattern.findall(text)
        T.append(code[0])
        Team_code.append(T)
 
    return Team_code

fixtures = weekly_fixtures()
if not fixtures:
    print('No match fixtures in this week \n visit:- https://www.premierleague.com/')
else:
    print(fixtures)
    

[['Crystal Palace', 'Newcastle United', '58989'], ['Brighton and Hove Albion', 'Liverpool', '58987'], ['Manchester City', 'Burnley', '58992'], ['Everton', 'Leeds United', '58990'], ['West Bromwich Albion', 'Sheffield United', '58994'], ['Southampton', 'Manchester United', '58993'], ['Chelsea', 'Tottenham Hotspur', '58988'], ['Arsenal', 'Wolverhampton Wanderers', '58986'], ['Leicester City', 'Fulham', '58991'], ['West Ham United', 'Aston Villa', '58995']]


[2]

In [83]:
# to get all scorer and assist for each team espn
import requests
import pandas as pd

from bs4 import BeautifulSoup

def stat(code):
    url = 'https://www.espn.in/football/team/stats/_/id/{}/league/ENG.1'.format(code)
    df = pd.read_html(url)
    return df
def goal_stat(code):
    goal = stat(code)[0]
    return goal
def assist_stat(code):
    assist = stat(code)[1]
    return assist
df = goal_stat(363)
df['Goal Percent'] = (df['G']/df['P'])*100
#df
df2 = assist_stat(363)
df2['Assist Percent'] = (df2['A']/df2['P'])*100
df

,RK,Name,P,G,Goal Percent
0,1.0,Timo Werner,9,4,44.444444
1,2.0,Tammy Abraham,8,3,37.500000
2,NaN,Kurt Zouma,8,3,37.500000
3,NaN,Jorginho,7,3,42.857143
4,5.0,Ben Chilwell,6,2,33.333333
5,6.0,Mason Mount,8,1,12.500000
6,NaN,Reece James,8,1,12.500000
7,NaN,Kai Havertz,7,1,14.285714
8,NaN,Callum Hudson-Odoi,5,1,20.000000
9,NaN,Hakim Ziyech,5,1,20.000000


In [44]:
# team code for goal number and assists espn
import requests
import pandas as pd
from bs4 import BeautifulSoup

r = requests.get('https://www.espn.in/football/league/_/name/eng.1/')
soup = BeautifulSoup(r.content, 'html.parser')
teams = soup.select('tr a')
for team in teams:
    link = team.get('href')
    pattern = re.compile(r'\d+')
    match = pattern.findall(link)
    print(team.text+','+match[0])

Tottenham Hotspur,367
Liverpool,364
Chelsea,363
Leicester City,375
Southampton,376
Everton,368
Aston Villa,362
West Ham United,371
Wolverhampton Wanderers,380
Manchester United,360
Crystal Palace,384
Arsenal,359
Manchester City,382
Leeds United,357
Newcastle United,361
Brighton & Hove Albion,331
Burnley,379
Fulham,370
West Bromwich Albion,383
Sheffield United,398


In [15]:
# premire leaugue standings

url = 'https://www.premierleague.com/'
def standing():
    df = pd.read_html(url)
    return df[0]
standing()

,Pos,Club,Pl,GD,Pts
0,1 Previous Position 2,Spurs,9,12,20
1,2 Previous Position 3,Liverpool,9,5,20
2,3 Previous Position 5,Chelsea,9,12,18
3,4 Previous Position 1,Leicester,9,6,18
4,5 Previous Position 4,Southampton,9,4,17
5,6 Previous Position 7,Everton,9,3,16
6,7 Previous Position 6,Aston Villa,8,8,15
7,8 Previous Position 12,West Ham,9,5,14
8,9 Previous Position 9,Wolves,9,-1,14
9,10 Previous Position 14,Man Utd,8,-1,13


In [99]:
# finding fantasy stats (incomplete)


<html><head>
<title>Access Denied</title>
</head><body>
<h1>Access Denied</h1>
 
You don't have permission to access "http://int.soccerway.com/national/england/premier-league/20202021/regular-season/r59136/" on this server.<p>
Reference #18.ceebdfad.1606211275.e60d7a4
</p></body>
</html>

[]

True
